In [2]:
import pandas as pd
import numpy as np
import json
import re

df_2018 = pd.read_csv('itpas2_2018_contributions2candidates.csv')

C:\Users\lucky\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (10,11,12,18,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_2018.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,...,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,Unnamed: 22
0,C00177436,N,M2,P,2.020000e+17,24K,CCM,"YODER FOR CONGRESS, INC",OVERLAND PARK,KS,...,1172017.0,1500,C00472365,H0KS03137,40458934,1147350,NaN,NaN,4.02e+18,NaN
1,C00242271,N,M2,P,2.020000e+17,24K,CCM,KEVIN MCCARTHY FOR CONGRESS,BAKERSFIELD,CA,...,1182017.0,5000,C00420935,H6CA22125,74716003,1147780,NaN,CONTRIBUTION,4.02e+18,NaN
2,C00242271,N,M2,P,2.020000e+17,24K,CCM,TIBERI FOR CONGRESS,COLUMBUS,OH,...,1182017.0,5000,C00347492,H0OH12062,74716004,1147780,NaN,CONTRIBUTION,4.02e+18,NaN
3,C00242271,N,M2,P,2.020000e+17,24K,CCM,JEFF FLAKE FOR US SENATE INC,TEMPE,AZ,...,1182017.0,2500,C00347260,S2AZ00141,74716067,1147780,NaN,CONTRIBUTION,4.02e+18,NaN
4,C00051979,N,M2,P,2.020000e+17,24K,CCM,KAINE FOR VIRGINIA,ALEXANDRIA,VA,...,1232017.0,1000,C00495358,S2VA00142,8481131,1147789,NaN,CONTRIBUTION,4.02e+18,NaN


In [3]:
del pcap_data

NameError: name 'pcap_data' is not defined

In [4]:
#loading info about candidates to get party info
df_can = pd.read_csv('cands18_fec.txt',sep="|", header=None)

In [5]:
df_can.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,NaN,2018,",",H6FL08221,",",N00028542,",",D J Mauro (I),",",I,...,,",",,",",,",",3N,",",,NaN
1,NaN,2018,",",H6NC09200,",",N00035491,",",Mark Harris (R),",",R,...,,",",,",",O,",",RO,",",,NaN
2,NaN,2018,",",H4NE01163,",",N00035619,",",Dennis Crawford (D),",",D,...,,",",Y,",",C,",",DL,",",,NaN
3,NaN,2018,",",H0IN01127,",",N00041294,",",Arturas Kerelis (D),",",D,...,,",",,",",,",",DN,",",,NaN
4,NaN,2018,",",H8AL02171,",",N00041295,",",Barry Moore (R),",",R,...,,",",Y,",",C,",",RL,",",,NaN


In [6]:
#subset to relevant columns
src_dst = df_2018[['CMTE_ID', 'CAND_ID', 'STATE','TRANSACTION_AMT']]

In [ ]:
src_dst.head()

In [7]:
src_dst.shape

(237090, 4)

Aggregate Records

In [7]:
grouped_src_dst = src_dst.groupby(['CMTE_ID', 'CAND_ID','STATE'])['TRANSACTION_AMT'].sum().reset_index()
#grouped_src_dst['TRANSACTION_AMT'].agg(np.sum)
grouped_src_dst.head()

,CMTE_ID,CAND_ID,STATE,TRANSACTION_AMT
0,C00000059,H0KS03137,KS,5000
1,C00000059,H0MO06073,MO,6000
2,C00000059,H2MA09072,MA,2500
3,C00000059,H2NC11080,NC,6000
4,C00000059,H2PA17079,PA,2000


In [10]:
grouped_src_dst.shape

(125932, 4)

In [8]:
s = grouped_src_dst['TRANSACTION_AMT']
s.isnull().values.any()

False

Format records

In [9]:
links_list = list(grouped_src_dst.apply(lambda row: {"source": row['CMTE_ID'], "target": row['CAND_ID'], "value": row['TRANSACTION_AMT']}, axis=1))

In [116]:
links_list[1:1000]

[{'source': 'C00000059', 'target': 'H0MO06073', 'value': 6000},
 {'source': 'C00000059', 'target': 'H2MA09072', 'value': 2500},
 {'source': 'C00000059', 'target': 'H2NC11080', 'value': 6000},
 {'source': 'C00000059', 'target': 'H2PA17079', 'value': 2000},
 {'source': 'C00000059', 'target': 'H4MO05234', 'value': 7000},
 {'source': 'C00000059', 'target': 'H4MO08162', 'value': 3000},
 {'source': 'C00000059', 'target': 'H6FL13148', 'value': 1000},
 {'source': 'C00000059', 'target': 'H6KS01179', 'value': 1000},
 {'source': 'C00000059', 'target': 'H6MD07160', 'value': 5000},
 {'source': 'C00000059', 'target': 'H8GA06286', 'value': 2500},
 {'source': 'C00000059', 'target': 'H8MO09153', 'value': 3000},
 {'source': 'C00000059', 'target': 'H8VA11062', 'value': 2500},
 {'source': 'C00000059', 'target': 'H8WI01024', 'value': 3500},
 {'source': 'C00000059', 'target': 'S2MT00096', 'value': 2000},
 {'source': 'C00000059', 'target': 'S2ND00099', 'value': 3000},
 {'source': 'C00000059', 'target': 'S4OK

In [11]:
#grouped_src_dst2 = src_dst.groupby(['CMTE_ID', 'STATE'])
#grouped_src_dst2.head()
df_nodes_C = pd.DataFrame(src_dst.CMTE_ID.unique())
df_nodes_C["group"] = "Committee"
df_nodes_C.head()

,0,group
0,C00177436,Committee
1,C00242271,Committee
2,C00051979,Committee
3,C00478099,Committee
4,C00280222,Committee


In [216]:
df_nodes_P = pd.DataFrame(src_dst.CAND_ID.unique())
df_nodes_P["group"] = "Politician"
df_nodes_P.head()

,0,group
0,H0KS03137,Politician
1,H6CA22125,Politician
2,H0OH12062,Politician
3,S2AZ00141,Politician
4,S2VA00142,Politician


In [162]:
df_nodes_P.shape

(1768, 2)

In [218]:
df_nodes_P[df_nodes_P[0]=='H6CO06229']

,0,group
1686,H6CO06229,Politician


In [219]:
df_nodes_P = pd.merge(df_nodes_P, df_can, left_on=0, right_on=3, how='left')
df_nodes_P.head()

,key_0,0_x,group,0_y,1,2,3,4,5,6,...,15,16,17,18,19,20,21,22,23,24
0,H0KS03137,H0KS03137,Politician,NaN,2018.0,",",H0KS03137,",",N00031502,",",...,Y,",",Y,",",I,",",RI,",",,NaN
1,H6CA22125,H6CA22125,Politician,NaN,2018.0,",",H6CA22125,",",N00028152,",",...,Y,",",Y,",",I,",",RI,",",,NaN
2,H0OH12062,H0OH12062,Politician,NaN,2018.0,",",H0OH12062,",",N00009699,",",...,,",",,",",I,",",RI,",",,NaN
3,S2AZ00141,S2AZ00141,Politician,NaN,2018.0,",",S2AZ00141,",",N00009573,",",...,,",",Y,",",I,",",RI,",",,NaN
4,S2VA00142,S2VA00142,Politician,NaN,2018.0,",",S2VA00142,",",N00033177,",",...,Y,",",Y,",",I,",",DI,",",,NaN


In [204]:
df_nodes_P.shape

(1768, 28)

In [221]:
df_nodes_P[df_nodes_P['key_0']=='H6CO06229']

,key_0,0_x,group,0_y,1,2,3,4,5,6,...,15,16,17,18,19,20,21,22,23,24
1686,H6CO06229,H6CO06229,Politician,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [222]:
df_nodes_P = df_nodes_P[['key_0',9]]
df_nodes_P.head()

,key_0,9
0,H0KS03137,R
1,H6CA22125,R
2,H0OH12062,R
3,S2AZ00141,R
4,S2VA00142,D


In [223]:
df_nodes_P.columns = [0 if x=='key_0' else x for x in df_nodes_P.columns]
df_nodes_P.columns = ['group' if x== 9 else x for x in df_nodes_P.columns]
df_nodes_P.head()

,0,group
0,H0KS03137,R
1,H6CA22125,R
2,H0OH12062,R
3,S2AZ00141,R
4,S2VA00142,D


In [224]:
df_nodes_P.shape

(1768, 2)

In [225]:
df_nodes_P.groupby('group')[0].count()

group
3         7
D       908
I        12
L         6
NM01      1
R       703
U         1
Name: 0, dtype: int64

In [227]:
df_nodes_P[df_nodes_P[0] == 'H6CO06229']
#df_nodes_P[df_nodes_P['key_0']=='H6CO06229']

,0,group
1686,H6CO06229,NaN


In [246]:
df_nodes_P['group'] = df_nodes_P['group'].fillna("TBD")

In [247]:
df_nodes_P[df_nodes_P[0] == 'H6CO06229']
#df_nodes_P[df_nodes_P['key_0']=='H6CO06229']

,0,group
1686,H6CO06229,TBD


In [248]:
df_nodes = pd.concat([df_nodes_P,df_nodes_C])
df_nodes.head()

,0,group
0,H0KS03137,R
1,H6CA22125,R
2,H0OH12062,R
3,S2AZ00141,R
4,S2VA00142,D


In [249]:
df_nodes.groupby('group')[0].count()

group
3               7
Committee    4315
D             908
I              12
L               6
NM01            1
R             703
TBD           129
U               1
Name: 0, dtype: int64

In [250]:
len(df_nodes)

6083

In [255]:
df_nodes.head()

,0,group
0,H0KS03137,R
1,H6CA22125,R
2,H0OH12062,R
3,S2AZ00141,R
4,S2VA00142,D


In [251]:
df_nodes['group'][pd.isna(df_nodes['group'])] = "TBD"

In [256]:
df_nodes[df_nodes[0 == 'H6CO06229']]

ValueError: cannot index with vector containing NA / NaN values

In [109]:
df_nodes = df_nodes.dropna()

In [254]:
s = df_nodes['group']
s.isnull().values.any()

False

In [257]:
node_list = list(df_nodes.apply(lambda row: {"name": row[0], "group": row['group']}, axis=1))

In [258]:
len(node_list)

6083

In [108]:
#node_list = list(grouped_src_dst.apply(lambda row: {"name": row['CAND_ID'], "name": row['CMTE_ID'],"group": row['STATE']}, axis=1))

In [151]:
node_list[-5:]

[{'group': 'Committee', 'name': 'C00678920'},
 {'group': 'Committee', 'name': 'C90017633'},
 {'group': 'Committee', 'name': 'C90017690'},
 {'group': 'Committee', 'name': 'C90017799'},
 {'group': 'Committee', 'name': 'C00130658'}]

In [75]:
node_list

[{'group': 'R', 'name': 'H0KS03137'},
 {'group': 'R', 'name': 'H6CA22125'},
 {'group': 'R', 'name': 'H0OH12062'},
 {'group': 'R', 'name': 'S2AZ00141'},
 {'group': 'D', 'name': 'S2VA00142'},
 {'group': 'R', 'name': 'H0VA02118'},
 {'group': 'R', 'name': 'S0WI00197'},
 {'group': 'D', 'name': 'S6VA00093'},
 {'group': 'R', 'name': 'S6UT00063'},
 {'group': 'R', 'name': 'H0IN02125'},
 {'group': 'R', 'name': 'S0FL00338'},
 {'group': 'R', 'name': 'H2AZ08102'},
 {'group': 'R', 'name': 'H4FL26038'},
 {'group': 'R', 'name': 'H2IN05082'},
 {'group': 'R', 'name': 'H0WA03187'},
 {'group': 'R', 'name': 'H0IL11052'},
 {'group': 'R', 'name': 'H0AL02087'},
 {'group': 'R', 'name': 'H0PA06076'},
 {'group': 'R', 'name': 'H0NY29054'},
 {'group': 'R', 'name': 'H8FL16022'},
 {'group': 'R', 'name': 'H0PA03271'},
 {'group': 'R', 'name': 'H0WI07051'},
 {'group': 'D', 'name': 'H4WI04183'},
 {'group': 'R', 'name': 'S6LA00300'},
 {'group': 'R', 'name': 'H6OR02116'},
 {'group': 'D', 'name': 'H8LA02054'},
 {'group': '

In [259]:
json_prep = {"nodes":node_list, "links":links_list}

json_prep.keys()
   # dict_keys(['links', 'nodes'])

dict_keys(['nodes', 'links'])

In [260]:
json_dump = json.dumps(json_prep, indent=1, sort_keys=True)

In [261]:
filename_out = 'test_export_party.json'
json_out = open(filename_out,'w')
json_out.write(json_dump)
json_out.close()